# 01 — Data Loading and Initial Understanding (CIC-IDS-2017)

## What this notebook demonstrates
This notebook is intentionally **not** about fancy charts or modeling.

It focuses on the boring but important part:
- **Load the raw CSVs safely** (day/scenario split)
- **Understand what’s inside** (columns, datatypes, labels)
- **Basic quality checks** (missing values, duplicates, structural quirks)
- **Reproducible workflow** (portable paths + shared inspection helpers)

## Dataset note (plain English)
CIC-IDS-2017 starts as **network packet captures (PCAP)**. Those packets are converted into
**flow records** (one row ≈ one bidirectional “conversation” between two endpoints), and each row
has **summary features** like duration, packet counts, bytes, and timing statistics.

In this repo:
- Raw files live in `data/raw/`
- Processed/cleaned outputs (later) will live in `data/processed/`

## Column cheat-sheet 
You’ll see columns like:
- **Destination Port**: what service/app the traffic likely targets (e.g., 80/443)
- **Flow Duration**: how long the flow lasted
- **Total Fwd/Bwd Packets & Lengths**: volume in each direction  
  (*Fwd = initiator → responder, Bwd = responder → initiator*)
- **IAT (Inter-Arrival Time)**: gaps between packets (mean/std/max/min)
- **TCP Flag Counts (SYN/ACK/RST/FIN …)**: handshake/reset patterns that can hint at attacks
- **Active/Idle stats**: bursty behavior vs pauses
- **Label**: ground-truth class (Monday is mostly baseline `BENIGN`)


This notebook performs initial inspection of the CIC-IDS-2017 dataset.
The goal is to validate dataset integrity, understand schema and labels,
and identify structural characteristics and potential data quality issues
before any downstream analysis.

In [62]:
from pathlib import Path
import pandas as pd
import sys

# Add repo root to Python path so `import src...` works no matter where notebook runs from
REPO_ROOT = Path.cwd().parent  # notebooks/ -> repo root
if str(REPO_ROOT) not in sys.path:
    sys.path.insert(0, str(REPO_ROOT))

print("Repo root added to path:", REPO_ROOT)

from src.utils import (
    list_csv_files,
    pick_file,
    load_peek,
    basic_audit,
    numeric_summary,
    find_suspicious_columns
)

# Display settings help during presentations
pd.set_option("display.max_columns", 120)
pd.set_option("display.width", 140)
pd.set_option("display.max_colwidth", 60)

DATA_RAW_DIR = Path("../data/raw")


Repo root added to path: c:\Users\Nimish\Desktop\cyber-log-analysis\cyber-log-analysis


## 1) Data acquisition (reproducible discovery)
Instead of hardcoding file paths, we discover all CIC-IDS-2017 CSV files in `data/raw/`.
This makes the notebook portable and avoids "works on my machine" problems.

We list the files and their sizes as a basic sanity check.


In [63]:
files = list_csv_files(DATA_RAW_DIR)
files


,file,size_mb
0,Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv,73.55
1,Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv,73.34
2,Friday-WorkingHours-Morning.pcap_ISCX.csv,55.62
3,Monday-WorkingHours.pcap_ISCX.csv,168.73
4,Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv,79.25
5,Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv,49.61
6,Tuesday-WorkingHours.pcap_ISCX.csv,128.82
7,Wednesday-workingHours.pcap_ISCX.csv,214.74


## 2) Pick a file for deep dive
CIC-IDS-2017 is split by day/scenario, so for the first pass we pick one file to inspect deeply.

Today, we'll look at **Monday** (mostly baseline traffic),
but this can be switched easily without changing the rest of the notebook.


In [64]:
csv_path = pick_file(DATA_RAW_DIR, preferred_name="Monday-WorkingHours.pcap_ISCX.csv")
csv_path.name


'Monday-WorkingHours.pcap_ISCX.csv'

## 3) Fast schema peek (first 5,000 rows)
These files can be large. A fast peek lets us:
- confirm we can parse the file correctly
- inspect columns and types
- identify label column and obvious issues early

This is a practical workflow: *don’t wait minutes just to discover something is broken.*


In [75]:
df = load_peek(csv_path, nrows=5000)

df.shape

(5000, 79)

## 4) First look (head/tail)
These quick previews verify the data "looks real" and helps us spot:
- weird parsing artifacts
- column shifting
- unexpected formatting issues


In [66]:
df.head(3)


,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Fwd Avg Bytes/Bulk,Fwd Avg Packets/Bulk,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,49188,4,2,0,12,0,6,6,6.0,0.0,0,0,0.0,0.0,3000000.0,500000.0,4.0,0.0,4,4,4,4.0,0.0,4,4,0,0.0,0.0,0,0,0,0,0,0,40,0,500000.0,0.0,6,6,6.0,0.0,0.0,0,0,0,0,1,1,0,0,0,9.0,6.0,0.0,40,0,0,0,0,0,0,2,12,0,0,329,-1,1,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
1,49188,1,2,0,12,0,6,6,6.0,0.0,0,0,0.0,0.0,12000000.0,2000000.0,1.0,0.0,1,1,1,1.0,0.0,1,1,0,0.0,0.0,0,0,0,0,0,0,40,0,2000000.0,0.0,6,6,6.0,0.0,0.0,0,0,0,0,1,1,0,0,0,9.0,6.0,0.0,40,0,0,0,0,0,0,2,12,0,0,329,-1,1,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2,49188,1,2,0,12,0,6,6,6.0,0.0,0,0,0.0,0.0,12000000.0,2000000.0,1.0,0.0,1,1,1,1.0,0.0,1,1,0,0.0,0.0,0,0,0,0,0,0,40,0,2000000.0,0.0,6,6,6.0,0.0,0.0,0,0,0,0,1,1,0,0,0,9.0,6.0,0.0,40,0,0,0,0,0,0,2,12,0,0,329,-1,1,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN


In [67]:
df.tail(3)


,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Fwd Avg Bytes/Bulk,Fwd Avg Packets/Bulk,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
4997,53,60578,1,1,51,150,51,51,51.0,0.0,150,150,150.0,0.0,3.318036e+03,33.015286,60578.000000,0.00000,60578,60578,0,0.0,0.0,0,0,0,0.0,0.0,0,0,0,0,0,0,20,20,16.507643,16.507643,51,150,84.0,57.157677,3267.0,0,0,0,0,0,0,0,0,1,126.00,51.0,150.0,20,0,0,0,0,0,0,1,51,1,150,-1,-1,0,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
4998,53,120500,1,1,48,144,48,48,48.0,0.0,144,144,144.0,0.0,1.593361e+03,16.597510,120500.000000,0.00000,120500,120500,0,0.0,0.0,0,0,0,0.0,0.0,0,0,0,0,0,0,32,32,8.298755,8.298755,48,144,80.0,55.425626,3072.0,0,0,0,0,0,0,0,0,1,120.00,48.0,144.0,32,0,0,0,0,0,0,1,48,1,144,-1,-1,0,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
4999,53,254,2,2,78,334,39,39,39.0,0.0,167,167,167.0,0.0,1.622047e+06,15748.031496,84.666667,63.50853,158,48,48,48.0,0.0,48,48,48,48.0,0.0,48,48,0,0,0,0,40,40,7874.015748,7874.015748,39,167,90.2,70.108487,4915.2,0,0,0,0,0,0,0,0,1,112.75,39.0,167.0,40,0,0,0,0,0,0,2,78,2,334,-1,-1,1,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN


## 5) Structural checks (column hygiene)
Common CSV problems:
- `Unnamed: 0` index columns
- duplicate column names
- hidden whitespace in column names

We detect these early so downstream code doesn't silently break.


In [68]:
find_suspicious_columns(df)


{'unnamed_columns': [], 'duplicate_columns': []}

## 6) Basic audit summary (Data Understanding + Quality)
This creates a compact first-pass “audit report”:
- shape
- duplicates
- memory footprint
- object columns (often unexpected)
- missingness top columns
- label distribution (if present)


In [69]:
audit = basic_audit(df, label_col="Label")
audit


AuditResult(shape=(5000, 79), n_duplicates=86, duplicate_rate=0.0172, memory_mb=3.2378501892089844, object_columns=['Label'], missing_top=Flow Bytes/s                   0.02
Flow Duration                  0.00
Destination Port               0.00
Total Backward Packets         0.00
Total Length of Fwd Packets    0.00
Total Length of Bwd Packets    0.00
Total Fwd Packets              0.00
Fwd Packet Length Max          0.00
Fwd Packet Length Min          0.00
Fwd Packet Length Std          0.00
Fwd Packet Length Mean         0.00
Bwd Packet Length Max          0.00
Bwd Packet Length Min          0.00
Bwd Packet Length Mean         0.00
Bwd Packet Length Std          0.00
dtype: float64, label_counts=Label
BENIGN    5000
Name: count, dtype: int64)

## 7) Labels: what are we classifying?
The `Label` column is the core of CIC-IDS-2017 classification tasks.

We check:
- does the label column exist?
- what classes appear?
- is the dataset imbalanced?

Imbalance matters because accuracy can be misleading if one class dominates.


In [70]:
if audit.label_counts is None:
    raise KeyError("Expected a 'Label' column but didn't find it in this file.")

audit.label_counts.head(20)


Label
BENIGN    5000
Name: count, dtype: int64

In [71]:
(audit.label_counts / audit.label_counts.sum() * 100).round(2).head(20)


Label
BENIGN    100.0
Name: count, dtype: float64

## 8) Missingness (top columns)
Missing values can:
- distort visualizations
- break models
- create bias when comparing across days

We report top missingness columns (%).


In [72]:
audit.missing_top


Flow Bytes/s                   0.02
Flow Duration                  0.00
Destination Port               0.00
Total Backward Packets         0.00
Total Length of Fwd Packets    0.00
Total Length of Bwd Packets    0.00
Total Fwd Packets              0.00
Fwd Packet Length Max          0.00
Fwd Packet Length Min          0.00
Fwd Packet Length Std          0.00
Fwd Packet Length Mean         0.00
Bwd Packet Length Max          0.00
Bwd Packet Length Min          0.00
Bwd Packet Length Mean         0.00
Bwd Packet Length Std          0.00
dtype: float64

## 9) Numeric sanity check (min/max/percentiles)
This catches obvious problems early:
- absurd min/max values
- huge scale differences (important for normalization)
- potential outliers that might dominate plots

This is not “deep stats”, it’s a quick health check.


In [73]:
numeric_summary(df, max_cols=20)


n:\Pycharm\Lib\site-packages\pandas\core\nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
n:\Pycharm\Lib\site-packages\pandas\core\nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


,count,mean,std,min,1%,50%,99%,max
Destination Port,5000.0,6.344973e+03,1.607810e+04,0.000000,53.000000,443.000000,5.466212e+04,6.462800e+04
Flow Duration,5000.0,2.663880e+07,3.989012e+07,0.000000,1.000000,562895.000000,1.183352e+08,1.199999e+08
Total Fwd Packets,5000.0,8.790360e+01,3.706303e+03,1.000000,1.000000,3.000000,2.340600e+02,2.197590e+05
Total Backward Packets,5000.0,1.122270e+02,4.923893e+03,0.000000,0.000000,2.000000,2.620400e+02,2.919220e+05
Total Length of Fwd Packets,5000.0,1.629051e+03,2.317702e+04,0.000000,0.000000,96.000000,1.992975e+04,1.323378e+06
Total Length of Bwd Packets,5000.0,2.364041e+05,1.102466e+07,0.000000,0.000000,117.000000,4.265714e+05,6.554530e+08
Fwd Packet Length Max,5000.0,3.104940e+02,9.377769e+02,0.000000,0.000000,51.000000,3.110300e+03,2.336000e+04
Fwd Packet Length Min,5000.0,1.601160e+01,2.663427e+01,0.000000,0.000000,0.000000,7.700000e+01,3.250000e+02
Fwd Packet Length Mean,5000.0,7.049216e+01,1.678970e+02,0.000000,0.000000,48.000000,5.860755e+02,4.141771e+03
Fwd Packet Length Std,5000.0,9.489541e+01,2.443242e+02,0.000000,0.000000,14.433757,1.030524e+03,5.199043e+03


## 10) What we learned 

**Reproducibility / Professional practice**
- Data is stored under `data/raw/` and discovered programmatically.
- Notebook uses shared helpers in `src/utils.py` for repeatable audits.

**Data understanding**
- We confirmed the schema and identified the label distribution.
- We identified dtypes and object columns that may need cleaning.

**Data quality**
- We measured missingness and duplicates (if any).
- We checked for structural issues like unnamed or duplicated columns.

## Next step (not done yet in this notebook)
In the next stage, we will:
- load multiple days/scenarios
- standardize schema across files
- write a processed dataset to `data/processed/` (likely Parquet?)
- begin visualization and feature exploration
